In [ ]:
# hide

%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [ ]:
# default_exp fastapi.main

## fastAPI Application Views

```
uvicorn will_it_saturate.fastapi.main:app --reload
```

In [ ]:
# export

from pathlib import Path

from fastapi import FastAPI
from fastapi.responses import FileResponse

In [ ]:
# export

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/pathinfo/{base_dir}")
def get_file_urls(base_dir: str):
    pass


@app.get("/{base}/{path}/{num}")
async def serve_file(base: str, path: str, num: int):
    file_path = str(Path(base) / path / str(num))
    return FileResponse(file_path)

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_core.ipynb.
Converted 01_django_views.ipynb.
Converted 01_fastapi_views.ipynb.
Converted 03_nginx_docker_server.ipynb.
Converted 03_servers.ipynb.
Converted 04_benchmark_clients.ipynb.
Converted 05_persistence.ipynb.
Converted 06_run_benchmark.ipynb.
Converted django_start_server.ipynb.
Converted index.ipynb.
Converted run_wrk_benchmark_test.ipynb.
Converted use_nginx_to_serve_files_test.ipynb.
